In [ ]:
import pandas as pd
import numpy as np
import glob
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Flatten, Input
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, f1_score, accuracy_score, precision_score, recall_score
from tensorflow.compat.v1.keras.layers import CuDNNLSTM

from google.colab import drive
drive.mount('/content/drive')

# Define the path to the directory containing the feature files
directory_path = '/content/drive/MyDrive/FYP_dataset/features'

# Get a list of all feature file paths in the directory
feature_files = glob.glob(directory_path + '/*.csv')

# Define lists to store the feature and target data
X = []
y_a = []
y_v = []

# Iterate over each feature file
for file in feature_files:
    # Read the feature file into a DataFrame
    df = pd.read_csv(file)

    # Extract the features and target variables
    features = df.drop(['Arousal_Value', 'Valence_Value', 'frameTime'], axis=1).values
    arousal = df['Arousal_Value'].values
    valence = df['Valence_Value'].values

    # Append the data to the lists
    X.append(features)
    y_a.append(arousal)
    y_v.append(valence)

# Concatenate the feature and target arrays
X = np.concatenate(X)
y_a = np.concatenate(y_a)
y_v = np.concatenate(y_v)

# Scale the features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Reshape the input data for LSTM
X_reshaped = X_scaled.reshape(X_scaled.shape[0], X_scaled.shape[1], 1)

# Train the TNN for feature selection
num_selected_features = 100  # Change this to the desired number of selected features

# Create TNN model
input_layer = Input(shape=(X_scaled.shape[1],))
tnn_output = Dense(num_selected_features, activation='relu')(input_layer)
selected_features = Dense(X_scaled.shape[1], activation='linear')(tnn_output)

tnn_model = Model(inputs=input_layer, outputs=selected_features)
tnn_model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=1e-5))

# Train TNN model
tnn_model.fit(X_scaled, X_scaled, epochs=10, batch_size=32)  # Adjust epochs and batch size as needed

# Apply TNN for feature selection
selected_features_scaled = tnn_model.predict(X_scaled)
selected_features_indices = np.argsort(selected_features_scaled[0])[::-1][:num_selected_features]

print("Selected feature indices:", selected_features_indices)
print("Number of selected features:", num_selected_features)

# Use selected features for the data
X_selected = X_scaled[:, selected_features_indices]

# Define the number of folds for cross-validation
num_folds = 5

# Initialize lists to store evaluation results
mse_a = []
mae_a = []
rmse_a = []
r2_a = []
f1_a = []
accuracy_a = []
precision_a = []
recall_a = []

# Perform tenfold cross-validation
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
for train_index, test_index in kf.split(X_selected):
    X_train, X_test = X_selected[train_index], X_selected[test_index]
    y_a_train, y_a_test = y_a[train_index], y_a[test_index]

    # Build the BiLSTM model for Arousal
    model_a = Sequential()
    model_a.add(Bidirectional(CuDNNLSTM(64, return_sequences=True), input_shape=(X_train.shape[1], 1)))
    model_a.add(Flatten())
    model_a.add(Dense(1))
    model_a.compile(loss='mse', optimizer=Adam(learning_rate=0.005))

    # Train the Arousal model
    model_a.fit(X_train, y_a_train, epochs=25, batch_size=32)

    # Evaluate the model on the test set
    predictions_a = model_a.predict(X_test)

    # Calculate additional evaluation metrics
    mse_a.append(mean_squared_error(y_a_test, predictions_a))
    mae_a.append(mean_absolute_error(y_a_test, predictions_a))
    rmse_a.append(np.sqrt(mean_squared_error(y_a_test, predictions_a)))
    r2_a.append(r2_score(y_a_test, predictions_a))

    # Convert regression predictions to binary labels
    threshold = 0
    binary_predictions_a = (predictions_a >= threshold).astype(int)
    y_a_test_binary = (y_a_test >= threshold).astype(int)

    # Calculate F1-score, accuracy, precision, and recall for binary classification
    f1_a.append(f1_score(y_a_test_binary, binary_predictions_a))
    accuracy_a.append(accuracy_score(y_a_test_binary, binary_predictions_a))
    precision_a.append(precision_score(y_a_test_binary, binary_predictions_a))
    recall_a.append(recall_score(y_a_test_binary, binary_predictions_a))

# Calculate average performance across all folds
average_mse_a = np.mean(mse_a)
average_mae_a = np.mean(mae_a)
average_rmse_a = np.mean(rmse_a)
average_r2_a = np.mean(r2_a)
average_f1_a = np.mean(f1_a)
average_accuracy_a = np.mean(accuracy_a)
average_precision_a = np.mean(precision_a)
average_recall_a = np.mean(recall_a)

print(f'Average Arousal MSE: {average_mse_a:.4f}')
print(f'Average Arousal MAE: {average_mae_a:.4f}')
print(f'Average Arousal RMSE: {average_rmse_a:.4f}')
print(f'Average Arousal R2 Score: {average_r2_a:.4f}')
print(f'Average Arousal F1 Score: {average_f1_a:.4f}')
print(f'Average Arousal Accuracy: {average_accuracy_a:.4f}')
print(f'Average Arousal Precision: {average_precision_a:.4f}')
print(f'Average Arousal Recall: {average_recall_a:.4f}')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/10
3317/3317 [==============================] - 10s 3ms/step - loss: 0.0256
Epoch 2/10
3317/3317 [==============================] - 11s 3ms/step - loss: 0.0033
Epoch 3/10
3317/3317 [==============================] - 10s 3ms/step - loss: 0.0026
Epoch 4/10
3317/3317 [==============================] - 9s 3ms/step - loss: 0.0022
Epoch 5/10
3317/3317 [==============================] - 9s 3ms/step - loss: 0.0019
Epoch 6/10
3317/3317 [==============================] - 10s 3ms/step - loss: 0.0016
Epoch 7/10
3317/3317 [==============================] - 10s 3ms/step - loss: 0.0015
Epoch 8/10
3317/3317 [==============================] - 9s 3ms/step - loss: 0.0013
Epoch 9/10
3317/3317 [==============================] - 10s 3ms/step - loss: 0.0012
Epoch 10/10
3317/3317 [==============================] - 6s 2ms/step
Selected feature indices: [ 97   3  11 105 141 10